# 총정리

In [1]:
!pip install transformers kss

     |████████████████████████████████| 2.9 MB 13.2 MB/s 
     |████████████████████████████████| 42.4 MB 1.7 MB/s 
     |████████████████████████████████| 895 kB 43.0 MB/s 
     |████████████████████████████████| 3.3 MB 53.7 MB/s 
     |████████████████████████████████| 56 kB 4.9 MB/s 
     |████████████████████████████████| 636 kB 62.7 MB/s 
     |████████████████████████████████| 168 kB 74.8 MB/s 
  Created wheel for kss: filename=kss-3.2.0-py3-none-any.whl size=42447995 sha256=3358d7549501164b68df08d1eefd26089ae696d3a92d1e05bd664b6bf23a6229
  Stored in directory: /root/.cache/pip/wheels/a1/47/9c/a5f83b5ab6096e3c4a33643fc553b26098c23e72b6539b86f4
  Created wheel for emoji: filename=emoji-1.6.0-py3-none-any.whl size=168256 sha256=d02576e3ee68630a6d687973fee7c835cf71112121500ad193dce1d433097a9e
  Stored in directory: /root/.cache/pip/wheels/f7/d7/74/c720aaf345a042b0c2d74361873258c5e8649b7f11b2ccce49
Successfully built kss emoji
  Attempting uninstall: pyyaml
    Found existing install

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import kss

[Korean Sentence Splitter]: Initializing Kss...


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 경로 설정
model_path = '/content/drive/MyDrive/kogpt2_models/'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
# base model 불러오기
from transformers import GPT2LMHeadModel
base_model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2',)
base_model.to(device)

[Korean Sentence Splitter]: Attempting to acquire lock 140160914706512 on /root/.cache/huggingface/transformers/13bb826cf24517d7849a701e02452715a67c5e560142be3d4735442b2a545809.6b384eec6effdd44287f67715cd55bd0dff2cf846d843b932b43ba7b632b8b1e.lock
[Korean Sentence Splitter]: Lock 140160914706512 acquired on /root/.cache/huggingface/transformers/13bb826cf24517d7849a701e02452715a67c5e560142be3d4735442b2a545809.6b384eec6effdd44287f67715cd55bd0dff2cf846d843b932b43ba7b632b8b1e.lock


Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

[Korean Sentence Splitter]: Attempting to release lock 140160914706512 on /root/.cache/huggingface/transformers/13bb826cf24517d7849a701e02452715a67c5e560142be3d4735442b2a545809.6b384eec6effdd44287f67715cd55bd0dff2cf846d843b932b43ba7b632b8b1e.lock
[Korean Sentence Splitter]: Lock 140160914706512 released on /root/.cache/huggingface/transformers/13bb826cf24517d7849a701e02452715a67c5e560142be3d4735442b2a545809.6b384eec6effdd44287f67715cd55bd0dff2cf846d843b932b43ba7b632b8b1e.lock
[Korean Sentence Splitter]: Attempting to acquire lock 140159339598544 on /root/.cache/huggingface/transformers/495b405e3742953dbcc56685d1560fa02a2d86fc50b891868990a4471b06c934.4ebf112d34c2c8fc657866680005d92d21859c52c0ef5e941fa640129b2f8f88.lock
[Korean Sentence Splitter]: Lock 140159339598544 acquired on /root/.cache/huggingface/transformers/495b405e3742953dbcc56685d1560fa02a2d86fc50b891868990a4471b06c934.4ebf112d34c2c8fc657866680005d92d21859c52c0ef5e941fa640129b2f8f88.lock


Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

[Korean Sentence Splitter]: Attempting to release lock 140159339598544 on /root/.cache/huggingface/transformers/495b405e3742953dbcc56685d1560fa02a2d86fc50b891868990a4471b06c934.4ebf112d34c2c8fc657866680005d92d21859c52c0ef5e941fa640129b2f8f88.lock
[Korean Sentence Splitter]: Lock 140159339598544 released on /root/.cache/huggingface/transformers/495b405e3742953dbcc56685d1560fa02a2d86fc50b891868990a4471b06c934.4ebf112d34c2c8fc657866680005d92d21859c52c0ef5e941fa640129b2f8f88.lock


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [5]:
# 토크나이저 불러오기
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
        "skt/kogpt2-base-v2",
        bos_token='<s>', eos_token='</s>', unk_token='<unk>',
        pad_token='<pad>', mask_token='<mask>')

[Korean Sentence Splitter]: Attempting to acquire lock 140321143217360 on /root/.cache/huggingface/transformers/fd8418e6675550cbca8ad6c102d717aa89372eb7a632ad3168300c7fed43491c.db074bfdd88bec54455de5ee2400efdbc64d4acf449a44d5f314e79c1eadc611.lock
[Korean Sentence Splitter]: Lock 140321143217360 acquired on /root/.cache/huggingface/transformers/fd8418e6675550cbca8ad6c102d717aa89372eb7a632ad3168300c7fed43491c.db074bfdd88bec54455de5ee2400efdbc64d4acf449a44d5f314e79c1eadc611.lock


Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

[Korean Sentence Splitter]: Attempting to release lock 140321143217360 on /root/.cache/huggingface/transformers/fd8418e6675550cbca8ad6c102d717aa89372eb7a632ad3168300c7fed43491c.db074bfdd88bec54455de5ee2400efdbc64d4acf449a44d5f314e79c1eadc611.lock
[Korean Sentence Splitter]: Lock 140321143217360 released on /root/.cache/huggingface/transformers/fd8418e6675550cbca8ad6c102d717aa89372eb7a632ad3168300c7fed43491c.db074bfdd88bec54455de5ee2400efdbc64d4acf449a44d5f314e79c1eadc611.lock
[Korean Sentence Splitter]: Attempting to acquire lock 140319537868240 on /root/.cache/huggingface/transformers/13bb826cf24517d7849a701e02452715a67c5e560142be3d4735442b2a545809.6b384eec6effdd44287f67715cd55bd0dff2cf846d843b932b43ba7b632b8b1e.lock
[Korean Sentence Splitter]: Lock 140319537868240 acquired on /root/.cache/huggingface/transformers/13bb826cf24517d7849a701e02452715a67c5e560142be3d4735442b2a545809.6b384eec6effdd44287f67715cd55bd0dff2cf846d843b932b43ba7b632b8b1e.lock


Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

[Korean Sentence Splitter]: Attempting to release lock 140319537868240 on /root/.cache/huggingface/transformers/13bb826cf24517d7849a701e02452715a67c5e560142be3d4735442b2a545809.6b384eec6effdd44287f67715cd55bd0dff2cf846d843b932b43ba7b632b8b1e.lock
[Korean Sentence Splitter]: Lock 140319537868240 released on /root/.cache/huggingface/transformers/13bb826cf24517d7849a701e02452715a67c5e560142be3d4735442b2a545809.6b384eec6effdd44287f67715cd55bd0dff2cf846d843b932b43ba7b632b8b1e.lock
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [8]:
model128_8000 = GPT2LMHeadModel.from_pretrained(model_path+'model128_8000').to(device)
model128_20000 = GPT2LMHeadModel.from_pretrained(model_path+'model128_20000').to(device)

model128_32000 = GPT2LMHeadModel.from_pretrained(model_path+'model128_32000').to(device)
model128_32000_ = GPT2LMHeadModel.from_pretrained(model_path+'model128_32000').to(device)
model128_32000_.eval()

model128_46560 = GPT2LMHeadModel.from_pretrained(model_path+'model128_46560').to(device)
model128_46560_ = GPT2LMHeadModel.from_pretrained(model_path+'model128_46560').to(device)
model128_46560_.eval()

model256_10000 = GPT2LMHeadModel.from_pretrained(model_path+'model256_10000').to(device)
model256_20000 = GPT2LMHeadModel.from_pretrained(model_path+'model256_20000').to(device)
model256_30000 = GPT2LMHeadModel.from_pretrained(model_path+'model256_30000').to(device)
model256_41400 = GPT2LMHeadModel.from_pretrained(model_path+'model256_41400').to(device)

In [ ]:
# text generation 함수
def generator(model):
    input_str = input("Input text: ")
    beam_outputs = model.generate(torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).to(device),
                                  do_sample=True,
                                  max_length=60,
                                  num_beams=3,
                                  num_return_sequences=3,
                                  top_k=50)
    out = dict()
    for i, beam_output in enumerate(beam_outputs):
        out[i] = tokenizer.decode(beam_output, skip_special_tokens=True)
    # 위에서 나오는 out은 대부분 max length만큼 문장이 생성되기 때문에 대부분 끝 문장이 중간에 잘려있다.

    text = [i for i in out.values()]
    text = kss.split_sentences(text)
    text = [i[:-1] for i in text]  # 마무리된 sentence까지만 가져온다.

    # out은 raw output dictionary,   text는 sentence split시키고 
    return formatted(text)

def formatted(text):
    print()
    print('-'*50)
    for i, txt in enumerate(text):
        num=0
        for sent in txt:
            if num == 0:
                if len(sent)>25:
                    print(f'{i+1}: {sent[:25]}\n   {sent[25:]}')
                elif len(sent)>50:
                    print(f'{i+1}: {sent[:25]}\n   {sent[25:50]}\n   {sent[50:]}')
                else:
                    print(f'{i+1}: {sent}')
                num += 1
            else:
                if len(sent)>25:
                    print(f'   {sent[:25]}\n   {sent[25:]}')
                elif len(sent)>50:
                    print(f'   {sent[:25]}\n   {sent[25:50]}\n   {sent[50:]}')
                else:
                    print(f'   {sent}')
        print()
        print()

In [16]:
generator(model128_46560)

Input text: 아이디어

--------------------------------------------------
1: 아이디어 선정 기술검토 서류작성 포스터 작성이
   었습니다
   팀원을 모집할 때 역할 배분을 고려하여 각각의
    과정을 맡을 수 있는 친구들과 함께하기로 했습니다
   그 후 각자 정해진 과정에 리더가 되어 팀 과
   제를 수행했습니다


2: 아이디어 선정 기술검토 서류작성 포스터 작성이
   었습니다
   팀원을 모집할 때 역할 배분을 고려하여 각각의
    과정을 맡을 수 있는 친구들과 함께하기로 했습니다
   그 후 각자 정해진 과정에 리더가 되어 팀 과
   제를 수행했습니다


3: 아이디어 선정 기술검토 서류작성 포스터 작성이
   었습니다
   팀원을 모집할 때 역할 배분을 고려하여 각각의
    과정을 맡을 수 있는 친구들과 함께하기로 했습니다
   그 후 각자 정해진 과정에 리더가 되어 팀 과
   제를 수행했습니다


